# ispy-jupyter

A demonstrator in which we read in NANOAOD and render it using three.js (via pythreejs) in a jupyter notebook

1. Read in nanoAOD and select
2. Create a renderer, scene, camera, etc.
3. Render an event

In [1]:
import json
import math
import uproot
import os

import awkward as ak
import ipywidgets as widgets
import numpy as np

from pythreejs import *
from IPython.display import display

## Part 1: nanoAOD

[Simulated dataset QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8 in NANOAODSIM format for 2016 collision data](https://opendata.cern.ch/record/63168)

In [2]:
infile_name = '397D1673-167A-CF46-9E79-D7069D9AC359.root'

if not (os.path.isfile(infile_name)): 
    !curl -O http://opendata.cern.ch/eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/270000/397D1673-167A-CF46-9E79-D7069D9AC359.root

In [3]:
infile = uproot.open(infile_name)

In [4]:
events = infile['Events']
#events.show()

[Dataset semantics](https://opendata.cern.ch/eos/opendata/cms/dataset-semantics/NanoAODSIM/63168/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8_doc.html)

## Part 2: three.js

Create a scene, renderer, controls, and lights

In [5]:
lp = 15.0

lights = [
    DirectionalLight(color='white', position=[-lp, lp, lp], intensity=1),
    DirectionalLight(color='white', position=[lp, -lp, -lp], intensity=1)
]

In [6]:
width = 800
height = 500

camera = PerspectiveCamera(
    position=[5, 5, 10],
    up=[0, 1, 0], 
    children=[lights],
    aspect=width/height
)

In [7]:
scene = Scene(
    background='#232323'
    #background='#ffffff'
)

In [8]:
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=width,
    height=height
)

In [9]:
def make_jet(pt, eta, phi):

    theta = 2*math.atan(pow(math.e, -eta))

    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    length = 1.10
    
    geometry = CylinderGeometry(
        radiusTop=0.3,
        radiusBottom=0.0,
        height=length,
        radialSegments=32,
        heightSegments=1,
        openEnded=True
    )

    length *= 0.5
    
    jet = Mesh(
        geometry=geometry,
        material=MeshBasicMaterial(
            color='#ffff00', 
            side='DoubleSide',
            transparent=True,
            opacity=0.5
        )
    )

    jet.rotateZ(phi-math.pi/2)
    jet.rotateX(math.pi/2-theta)
    jet.position = (dir*length).tolist()
    jet.name = 'Jet'

    jet.props = {
        'pt': pt,
        'eta': eta,
        'phi': phi
    }
    
    return jet

make_jets = np.vectorize(make_jet)

For now just draw electrons and muons as arrows

In [10]:
def make_muon(pt, eta, phi, charge):

    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    
    muon = ArrowHelper(
      dir=dir.tolist(),
      origin=[0, 0, 0],
      length=pt*0.1,
      color='#ff0000',
      headLength=0.2,
      headWidth=0.1
    )

    muon.name = 'Muon'
    muon.props = {
        'pt': pt,
        'eta': eta,
        'phi': phi,
        'charge': charge
    }
    
    return muon

make_muons = np.vectorize(make_muon)

In [11]:
def make_electron(pt, eta, phi, charge):

    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    
    electron = ArrowHelper(
      dir=dir.tolist(),
      origin=[0, 0, 0],
      length=pt*0.1,
      color='#19ff19',
      headLength=0.2,
      headWidth=0.1
    )

    electron.name = 'Electron'
    electron.props = {
        'pt': pt,
        'eta': eta,
        'phi': phi,
        'charge': charge
    }
    
    return electron

make_electrons = np.vectorize(make_electron)

In [12]:
def make_met(pt, phi):

    px = math.cos(phi)
    py = math.sin(phi)
    
    dir = np.array([px,py,0])
    dir /= np.linalg.norm(dir)
    d = 1.24
    length = pt*0.1
    length = 5 if length+d > 5 else length
    
    met = ArrowHelper(
      dir=dir.tolist(),
      origin=[px*d, py*d, 0],
      length=length,
      color='#ff00ff',
      headLength=0.2,
      headWidth=0.2
    )

    met.name = 'MET'
    met.props = {
        'pt': pt,
        'phi': phi
    }

    return met

For control over cone properties and line thickness try this method. It doesn't seem to help with picking yet (although the head of the arrow can be picked).

In [13]:
def make_thick_arrow(pt, phi):

    px = math.cos(phi)
    py = math.sin(phi)
    
    dir = np.array([px,py,0])
    dir /= np.linalg.norm(dir)
    d = 1.24
    length = pt*0.1
    length = 5 if length+d > 5 else length

    lg = LineSegmentsGeometry(
        positions=[
            [
                (dir*d).tolist(), 
                (dir*(length+d-0.2)).tolist()
            ]
        ],
    )
    
    lm = LineMaterial(linewidth=3, color='#ff00ff')
    line = LineSegments2(lg, lm)
    line.name = 'MET'
    
    cg = CylinderGeometry(
        radiusTop=0.0,
        radiusBottom=0.1,
        height=0.2,
        radialSegments=32,
        heightSegments=1,
        openEnded=True
    )

    cone = Mesh(cg, MeshBasicMaterial(color='#ff00ff'))
    cone.rotateZ(phi-math.pi/2)
    cone.position = (dir*(length+d-0.2)).tolist()
    
    # We can pick MET when we pick the cone 
    # but some reason lines aren't able to be picked.
    #cone.name = 'MET'
    
    met = Object3D(
        name='MET',
        children=(line,cone)
    )

    met.props = {
        'pt': pt,
        'phi': phi
    }
    
    return met

In [14]:
def make_sv(x,y,z):

    geometry = SphereGeometry(
        radius=0.01,
        widthSegments=32,
        heightSegments=32
    )

    vertex = Mesh(
        geometry,
        MeshBasicMaterial(color='#ff6600')
    )

    vertex.name = 'SV'
    vertex.position = [0.01*x,0.01*y,0.01*z]
    
    return vertex

make_svs = np.vectorize(make_sv)

def make_pv(position):

    geometry = SphereGeometry(
        radius=0.01,
        widthSegments=32,
        heightSegments=32
    )

    vertex = Mesh(
        geometry,
        MeshBasicMaterial(color='#ffff00')
    )

    vertex.name = 'PV'
    vertex.position = position

    return vertex

Make an EB-like geometry for context

In [15]:
def make_eb():

    geometry = CylinderGeometry(
        radiusTop=1.12, 
        radiusBottom=1.24, 
        height=6.0, 
        radialSegments=64, 
        heightSegments=1, 
        openEnded=True, 
        thetaStart=0, 
        thetaLength=2*math.pi
    )

    eb = Mesh(
        geometry,
        MeshBasicMaterial(
            color='#7fccff',
            wireframe=True,
            transparent=True,
            opacity=0.2
        )
    )

    eb.rotateX(math.pi/2)
    eb.name = 'EB'
    
    return eb

## Part 3: Render events

In [16]:
df = events.arrays(
    [
        'run', 'event', 'luminosityBlock',
        'nJet', 'Jet_pt', 'Jet_eta', 'Jet_phi',
        'MET_pt', 'MET_phi',
        'nPhoton', 'Photon_pt', 'Photon_eta', 'Photon_phi',
        'nMuon', 'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_charge',
        'nElectron', 'Electron_pt', 'Electron_eta', 'Electron_phi', 'Electron_charge',
        'nSV', 'SV_x', 'SV_y', 'SV_z',
        'PV_x', 'PV_y', 'PV_z'
    ],
    library='ak'
)

In [17]:
df

<Array [{run: 1, event: 4065004, ...}, ...] type='2000 * {run: uint32, even...'>

Make information boxes for run/event/ls information and picked object information

In [18]:
info_vbox = widgets.VBox()
event_info = widgets.HTML()

info_vbox.children = (event_info,) + info_vbox.children

In [19]:
pick_vbox = widgets.VBox()
pick_info = widgets.HTML(
    value='Object info: '
)

pick_vbox.children = (pick_info,) + pick_vbox.children

In [20]:
def show_event_info(event):
    out_text = f"Run/Event/LS : {event['run']}/{event['event']}/{event['luminosityBlock']}"    
    event_info.value = out_text

In [21]:
def add_event(event):
    
    scene.children = []

    show_event_info(event)
    
    met = make_met(
        event['MET_pt'],
        event['MET_phi']
    )

    met2 = make_thick_arrow(
        event['MET_pt'],
        event['MET_phi']
    )
    
    #scene.add(met)
    scene.add(met2)
    
    if event['nJet'] > 0:
    
        jets = make_jets(
            event['Jet_pt'],
            event['Jet_eta'],
            event['Jet_phi']
        )

        scene.add(jets)

    if event['nElectron'] > 0:

        electrons = make_electrons(
            event['Electron_pt'],
            event['Electron_eta'],
            event['Electron_phi'],
            event['Electron_charge']
        )

        scene.add(electrons)

    if event['nMuon'] > 0:
        
        muons = make_muons(
            event['Muon_pt'],
            event['Muon_eta'],
            event['Muon_phi'],
            event['Muon_charge']
        )

        scene.add(muons)
        
    if event['nSV'] > 0:
        
        svs = make_svs(
            event['SV_x'],
            event['SV_y'],
            event['SV_z']
        )
         
        scene.add(svs)

    pv = make_pv([
        0.01*event['PV_x'],
        0.01*event['PV_y'],
        0.01*event['PV_z']
    ])

    scene.add(pv)
    
    scene.add(make_eb())

In [22]:
renderer.render(scene, camera)

current_event = 0
max_events = len(df)-1

prev_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Previous Event',
    icon='step-backward'
)

next_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Next Event',
    icon='step-forward'
)

output = widgets.Output()

def prev_button_clicked(b):
    with output:
        global current_event
        if current_event > 0:
            current_event -= 1
            add_event(df[current_event])
        
def next_button_clicked(b):
    with output:
        global current_event
        if current_event < max_events:
            current_event += 1
            add_event(df[current_event])

prev_button.on_click(prev_button_clicked)
next_button.on_click(next_button_clicked)

Test a picker. For now just select jets and on hover change the color and display the jet properties. This could get very complicated very quickly so need to think about how to expand this to other objects.

In [23]:
picker = Picker(
    controlling=scene, 
    event='mousemove',
    lineThreshold=0.1,
    pointThreshold=0.1,
    all=True
)

renderer.controls = renderer.controls + [picker]
last_hovered_object = None

def on_object_hovered(change):
    
    global last_hovered_object
    global pick_info
    
    hovered_object = change['new']
    pick_info.value = 'Object info: '

    # This is what gets done when the mouse moves off from the object
    if last_hovered_object and last_hovered_object != hovered_object:
        if last_hovered_object.name == 'Jet':
            last_hovered_object.material.color = '#ffff00'
            pick_info.value = 'Object info: '
        if last_hovered_object.name == 'MET':
            last_hovered_object.material.color = '#ff00ff'
            pick_info.value = 'Object info: '

    # This is what gets done when the mouse hovers over the object
    if hovered_object:
        if hovered_object.name == 'Jet':
            hovered_object.material.color = '#ffffff'
            pick_info.value = f'Object info: Jet {json.dumps(hovered_object.props)}'
        if hovered_object.name == 'MET':
            hovered_object.material.color = '#ffffff'
            pick_info.value = f'Object info: Jet {json.dumps(hovered_object.props)}'
        
        last_hovered_object = hovered_object 
    else:
        last_hovered_object = None
    
picker.observe(on_object_hovered, names=['object'])

Add the current event and render

In [24]:
add_event(df[current_event])

display(widgets.HBox((prev_button, next_button)), output)
display(info_vbox)
display(pick_vbox)
display(renderer)

Output()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=([DirectionalLight(color='white', position=(-15.0, 15.0…